<h2> In-class transcript from Lecture 4, January 16, 2020</h2>

# Imports and defs for lecture

In [ ]:
# These are the standard imports for CS 111. 
# This list may change as the quarter goes on.

import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import sparse
from scipy import linalg as spla
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
%matplotlib inline

In [ ]:
def Usolve(U, y):
    """Backward solve an upper triangular system Ux = y for x
    Parameters: 
      U: the matrix, must be square, upper triangular, with nonzeros on the diagonal
      y: the right-hand side vector
    Output:
      x: the solution vector to U @ x == y
    """
    
    raise NotImplemented("you will write Usolve() as part of homework 2")
        
    return x

In [ ]:
def LUfactorNoPiv(A):
    """Factor a square matrix, A == L @ U (no partial pivoting)
    Parameters: 
      A: the matrix.
    Outputs (in order):
      L: the lower triangular factor, same dimensions as A, with ones on the diagonal
      U: the upper triangular factor, same dimensions as A
    """
    
    # Check the input
    m, n = A.shape
    assert m == n, 'input matrix A must be square'
    
    # Make a copy of the matrix that we will transform into L and U
    LU = A.astype(np.float64).copy()
    
    # Eliminate each column in turn
    for piv_col in range(n):
            
        # Update the rest of the matrix
        pivot = LU[piv_col, piv_col]
        assert pivot != 0., "pivot is zero, can't continue"
        for row in range(piv_col + 1, n):
            multiplier = LU[row, piv_col] / pivot
            LU[row, piv_col] = multiplier
            LU[row, (piv_col+1):] -= multiplier * LU[piv_col, (piv_col+1):]
            
    # Separate L and U in the result
    U = np.triu(LU)
    L = LU - U + np.eye(n)
    
    return (L, U)

In [ ]:
def LUfactor(A, pivoting = True):
    """Factor a square matrix with partial pivoting, A[p,:] == L @ U
    Parameters: 
      A: the matrix.
      pivoting: whether or not to do partial pivoting
    Outputs (in order):
      L: the lower triangular factor, same dimensions as A, with ones on the diagonal
      U: the upper triangular factor, same dimensions as A
      p: the permutation vector that permutes the rows of A by partial pivoting
    """
    
    # Check the input
    m, n = A.shape
    assert m == n, 'input matrix A must be square'
    
    # Initialize p to be the identity permutation
    p = np.array(range(n))
    
    # Make a copy of the matrix that we will transform into L and U
    LU = A.astype(np.float64).copy()
    
    # Eliminate each column in turn
    for piv_col in range(n):
        
        # Choose the pivot row and swap it into place
        if pivoting:
            piv_row = piv_col + np.argmax(np.abs(LU[piv_col:, piv_col]))
            assert LU[piv_row, piv_col] != 0., "can't find nonzero pivot, matrix is singular"
            LU[[piv_col, piv_row], :]  = LU[[piv_row, piv_col], :]
            p[[piv_col, piv_row]]      = p[[piv_row, piv_col]]
            
        # Update the rest of the matrix
        pivot = LU[piv_col, piv_col]
        assert pivot != 0., "pivot is zero, can't continue"
        for row in range(piv_col + 1, n):
            multiplier = LU[row, piv_col] / pivot
            LU[row, piv_col] = multiplier
            LU[row, (piv_col+1):] -= multiplier * LU[piv_col, (piv_col+1):]
            
    # Separate L and U in the result
    U = np.triu(LU)
    L = LU - U + np.eye(n)
    
    return (L, U, p)

In [ ]:
def Lsolve(L, b):
    """Forward solve a unit lower triangular system Ly = b for y
    Parameters: 
      L: the matrix, must be square, lower triangular, with ones on the diagonal
      b: the right-hand side vector
    Output:
      y: the solution vector to L @ y == b
    """
    
    # Check the input
    m, n = L.shape
    assert m == n, "matrix L must be square"
    assert np.all(np.tril(L) == L), "matrix L must be lower triangular"
    assert np.all(np.diag(L) == 1), "matrix L must have ones on the diagonal"
    
    # Make a copy of b that we will transform into the solution
    y = b.astype(np.float64).copy()
    
    # Forward solve
    for col in range(n):
        y[col+1:] -= y[col] * L[col+1:, col]
        
    return y

In [ ]:
def LUsolve(A, b):
    """Solve a linear system Ax = b for x by LU factorization with partial pivoting.
    Parameters: 
      A: the matrix.
      b: the right-hand side
    Outputs (in order):
      x: the computed solution
      rel_res: relative residual norm,
        norm(b - Ax) / norm(b)
    """
    
    # Check the input
    m, n = A.shape
    assert m == n, 'input matrix A must be square'
    
    # LU factorization
    L, U, p = LUfactor(A)
    
    # Forward and back substitution
    y = Lsolve(L,b[p])
    x = Usolve(U,y)
    
    # Residual norm
    rel_res = npla.norm(b - A@x) / npla.norm(b)
    
    return (x, rel_res)

# Lecture starts here

In [ ]:
A = np.round(100*np.random.rand(5,5))
print("A:", A)
print()
x = np.random.rand(5)
print('x:', x)
print()
b = A @ x
print('b:', b)

In [ ]:
xhat = npla.solve(A,b)
print("computed xhat:", xhat)

In [ ]:
error = x - xhat
print("error in xhat:", error)

In [ ]:
residual = b - A@xhat
print("residual:", residual)

In [ ]:
print("norm of residual:", npla.norm(residual))

In [ ]:
print("relative norm of residual:", npla.norm(residual) / npla.norm(b))

In [ ]:
npla.norm(b)

In [ ]:
# Example of a unit lower triangular matrix

L = np.array([[1,0,0,0],[.5,1,0,0],[0,.5,1,0],[-.5,-.5,0,1]])
print("L:", L)
print()
b = np.array([17. ,  2.5, -7. , 10.5])
print("b:", b)

In [ ]:
L @ np.array([17,-6,-4,16])

In [ ]:
# But LU factorization (without pivoting) fails if it encounters a zero pivot

A = np.array([[0, 1], [1, 2]])
L,U = LUfactorNoPiv(A)


In [ ]:
# Our 2-by-2 counterexample again, with partial pivoting this time

print('A:\n', A)
L, U, p = LUfactor(A)
print('\nL:\n', L)
print('\nU:\n', U)
print('\np: ', p)

In [ ]:
# A larger example of LU with partial pivoting

A = np.round(20*np.random.rand(5,5))
print('matrix A:\n', A)
xorig = np.random.rand(5)
print('\noriginal x:', xorig)
b = A @ xorig
print('\nright-hand side b:', b)

In [ ]:
# Factor the larger example

L, U, p = LUfactor(A,pivoting=True)
print("norm of difference between L times U and permuted A:", npla.norm( L@U - A[p,:]))

<H2>Appendix to Lecture: Permutation vectors and permutation matrices</H2>

In the output of LUfactor above, p is a "permutation vector", that is, a vector containing the elements of range(n) (that is, 0 through n-1) in some permuted order. In this case, p records the permutation of the rows of A that was caused by the pivot swaps in the factorization. That's the reason L @ U is equal to A[p, :] (to 15 significant digits), as shown by the output on the line above.

Let's look at p, L, and U. Note that, as expected, p is a permutation of range(5), L is unit lower triangular, and U is upper triangular. Notice also that every element of L is less than or equal to 1 in absolute value. This is _always_ true with partial pivoting, because the row swap guarantees that the pivot is the largest-magnitude element in the remaining part of the column, and therefore the multipliers can't be larger than 1.

In [ ]:
print('p =',p)
print()
print('L =\n', L)
print()
print('U =\n', U)

How do we solve A@x = b using L, U, and p? Every time we swap two rows of the matrix, we need to swap the same two rows of the right-hand side b in order for the equations to be the same. Here's how that works.


In [ ]:
y = Lsolve(L,b[p])
print("y:", y)
# x = Usolve(U,y) # Use this after you've written Usolve() for Homework 2
x = npla.solve(U,y) # Use this before you write Usolve() for Homework 2
print("\nx:", x)
print("\nrelative residual norm:", npla.norm(b - A @ x)/npla.norm(b))

It's natural in numpy (or in Matlab) to permute a vector or the rows or columns of a matrix by indexing with a permutation vector. In linear algebra, though, it's traditional to permute things by multiplying with a _permutation matrix_. A permutation matrix is a square matrix that is all 0s except for exactly one 1 in every row and column. Here's an example.

In [ ]:
P = np.eye(5)[p,:]
print('P =\n',P)
print()
print('P @ A =\n', P @ A)
print()
print('p =',p)
print()
print('A[p,:] =\n',A[p,:])

Multiplying A by P on the left permutes the rows of A, and this particular P does the same row permutation as the permutation vector p. Can you see how to read off the elements of p from the matrix P?

A good rule to remember is that <b>multiplying something on the left by A does something to the rows of A</b>, and <b>multiplying A by something on the right does something to the columns of A</b>. More specifically, the "something" it does to the rows or columns is forming a linear combination. We've already seen this in A @ x == b: Multiplying A on the right by x forms a linear combination of the columns of A that's equal to b. In P @ A, we get a different linear combination of the rows of A for each row of P, but the linear combinations are all easy: each one is just a single row of A. That's why P @ A permutes the rows of A.

So, we expect that multiplying by a permutation matrix on the right should permute the columns of A. That's exactly what happens, but there's a gotcha to be careful of. Watch the example:

In [ ]:
print('p =',p)
print()
print('A[:,p] =\n',A[:,p])
print()
print('P =\n',P)
print()
print('A @ P =\n', A @ P)

Sure enough, A[:, p] permuted the columns of A the same way A[p, :] permuted the rows. But A @ P permuted the columns differently! What went wrong?

It turns out that to apply a permutation to the columns of a matrix, you multiply on the right by the _transpose_ of the permutation matrix. Let's try that:

In [ ]:
print('p =',p)
print()
print('A[:,p] =\n',A[:,p])
print()
print('P transposed = P.T =\n',P.T)
print()
print('A @ P.T =\n', A @ P.T)

We'll see more permutation matrices later in the course.